In [72]:
# import sklearn packages for modelling
from sklearn import metrics
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
#from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [73]:
# All Package Imports
import pandas as pd
import numpy as np
import os
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import time
from datetime import datetime

In [74]:
#Importing Data
All_data = pd.read_csv("Covid_data.csv") 
print(len(All_data))
All_data['r_rows'] =np.where((All_data['data_type'] =='Train') &(All_data['t_flag'] ==1) & (All_data['t_10'] ==0),0,1)
All_data =All_data[All_data['r_rows'] ==1].reset_index(drop =True)
All_data =All_data.dropna(subset=['t_flag'], axis =0).reset_index(drop =True)
# List of columns 
All_data.columns

47825


Index(['DBR_in_percentage', 'Merchant_City', 'Merchant_Pincode',
       'Merchant_State', 'New_Model_Bands', 'age', 'app_downloaded_android',
       'app_downloaded_ios', 'aqb', 'assign_line_of_credit',
       ...
       'total_principal_pending', 'total_unreliable_imei',
       'twelve_month_sum_emi_overall', 'twelve_month_sum_overdue_overall',
       'user_created_at', 'verified_email', 'wiwo_ineligible_reason',
       'write_off_amount', 'write_off_past_12months', 'r_rows'],
      dtype='object', length=224)

In [75]:
len(All_data)

44162

In [76]:
# len(All_data)
All_data['disbursement_date']

0        2020-03-12
1        2020-03-14
2        2020-03-13
3        2020-03-16
4        2020-03-12
            ...    
44157    2020-03-21
44158    2020-03-21
44159    2020-03-21
44160    2020-03-21
44161    2020-03-21
Name: disbursement_date, Length: 44162, dtype: object

In [77]:
# List of numeric columns that will be used in model
print(All_data.shape)
useful_cols = ['avg_12_4_debit_count',
'avg_3_mon_income',
'avg_3_month_bounce_count',
'avg_6_mon_income',
'avg_6_month_bounce_count',
'avg_9_12_debit_amount',
'avg_9_12_debit_count',
'avg_9_12_debit_amount_per_count',
'avg_balance_3_month',
'avg_balance_6_month',
'avg_cashwithdrawl_6_month',
'avg_cc_spend_count',
'avg_diff_credit_debit',
'count_loan_overdues',
'count_loan_overdues_1m',
'count_name_match',
'data_type',
'credit_sum_avg',
'debit_sum_avg',
'diff_credit_debit',
'estimated_income_v2',
'kreditb_loan_approval_before_fa',
'kreditb_loan_overdue_before_fa',
'kreditbee_sum_paid_loans',
'last_mon_12_4_debit_count',
'last_mon_9_12_debit_amount',
'last_mon_income',
'last_month_bounce_count',
'loan_overdues',
'loan_overdues_1m',
'month_of_max_balance',
'number_of_application',
'number_of_devices_2w',
'primary_bank_avg_balance',
'primary_bank_avg_balance_3_mon',
'primary_bank_avg_balance_6_mon',
'primary_bank_max_balance',
'primary_bank_mon_max_balance',
'six_mon_avg_12_4_debit_count',
'total_contacts',
'total_credit_count',
'total_debit_count',
'total_income',              
'total_loan_approval_before_fa',
'total_loan_rejected',
'total_unreliable_imei',
'avg_six_monthly_emi',
'avg_three_monthly_emi',
'avg_twelve_monthly_emi',
'customer_first_name_messages',
'last_month_count_loans_overall',
'last_month_count_loans_pl',
'last_month_max_overall',
'last_month_max_pl',
'last_six_month_count_loans_pl',
'last_six_month_max_overall',
'last_six_month_max_pl',
'last_three_month_count_loans_pl',
'last_three_month_max_overall',
'last_three_month_max_pl',
'last_twelve_month_count_loans_pl',
'last_twelve_month_max_overall',
'last_twelve_month_max_pl',
'six_month_sum_emi_overall',
'six_month_sum_overdue_overall',
'three_month_sum_emi_overall',
'three_month_sum_overdue_overall',
'three_months_emi_sum_pl',
'twelve_month_sum_emi_overall',
'twelve_month_sum_overdue_overall',
'is_wonderland',
'age',
't_flag',
'device_present_age_in_days',
'is_digiscore_reliable',
'kissht_user_id']

(44162, 224)


In [78]:
# list of object columns (Check that any column which should have been numeric but is an object )
print(All_data.dtypes[All_data.dtypes==np.object])
All_data = All_data[useful_cols]
#All_data = All_data.replace(to_replace="#NAME?", value = None)

Merchant_City              object
app_downloaded_android     object
app_downloaded_ios         object
aqb                        object
assign_line_of_credit      object
                            ...  
user_created_at            object
verified_email             object
wiwo_ineligible_reason     object
write_off_amount           object
write_off_past_12months    object
Length: 111, dtype: object


In [79]:
numeric_cols = [
'avg_12_4_debit_count',
'avg_3_mon_income',
'avg_3_month_bounce_count',
'avg_6_mon_income',
'avg_6_month_bounce_count',
'avg_9_12_debit_amount',
'avg_9_12_debit_count',
'avg_9_12_debit_amount_per_count',
'avg_balance_3_month',
'avg_balance_6_month',
'avg_cashwithdrawl_6_month',
'avg_cc_spend_count',
'avg_diff_credit_debit',
'count_loan_overdues',
'count_loan_overdues_1m',  
'count_name_match',
'credit_sum_avg',
'debit_sum_avg',
'diff_credit_debit',
'estimated_income_v2',
'kreditb_loan_approval_before_fa',
'kreditb_loan_overdue_before_fa',
'kreditbee_sum_paid_loans',
'last_mon_12_4_debit_count',
'last_mon_9_12_debit_amount',
'last_mon_income',
'last_month_bounce_count',
'loan_overdues',
'loan_overdues_1m',
'month_of_max_balance',
'number_of_application',
'number_of_devices_2w',
'primary_bank_avg_balance',
'primary_bank_avg_balance_3_mon',
'primary_bank_avg_balance_6_mon',
'primary_bank_max_balance',
'primary_bank_mon_max_balance',
'six_mon_avg_12_4_debit_count',
'total_contacts',
'total_credit_count',
'total_debit_count',
'total_income',
'total_loan_approval_before_fa',
'total_loan_rejected',
'total_unreliable_imei',
'avg_six_monthly_emi',
'avg_three_monthly_emi',
'avg_twelve_monthly_emi',
'customer_first_name_messages',
'last_month_count_loans_overall',
'last_month_count_loans_pl',
'last_month_max_overall',
'last_month_max_pl',
'last_six_month_count_loans_pl',
'last_six_month_max_overall',
'last_six_month_max_pl',
'last_three_month_count_loans_pl',
'last_three_month_max_overall',
'last_three_month_max_pl',
'last_twelve_month_count_loans_pl',
'last_twelve_month_max_overall',
'last_twelve_month_max_pl',
'six_month_sum_emi_overall',
'six_month_sum_overdue_overall',
'three_month_sum_emi_overall',
'three_month_sum_overdue_overall',
'three_months_emi_sum_pl',
'twelve_month_sum_emi_overall',
'twelve_month_sum_overdue_overall',
'is_wonderland',
'age',
'device_present_age_in_days',
'is_digiscore_reliable']

In [80]:
# Remving t_10 column (as the numric columns will be used for training data )
All_data[numeric_cols] =All_data[numeric_cols].apply(pd.to_numeric, errors ='coerce')
All_data[numeric_cols] =All_data[numeric_cols].fillna(0)

In [81]:
Data_train =All_data[All_data["data_type"]=="Train"]
Data_test =All_data[All_data["data_type"]=="Test"]
Data_OOT =All_data[All_data["data_type"]=="OOT"]
# Data_OOT2 =All_data[All_data["tran_test_val_tag"]=="OOT2"]
Data_train_user_ids = Data_train["kissht_user_id"]
Data_test_user_ids = Data_test["kissht_user_id"]
Data_OOT_user_ids = Data_OOT["kissht_user_id"]
# Data_OOT2_user_ids = Data_OOT2["kissht_user_id"]
Data_test =Data_test.drop(["kissht_user_id","data_type"], axis=1)
Data_train =Data_train.drop(["kissht_user_id","data_type"], axis=1)
Data_OOT =Data_OOT.drop(["kissht_user_id","data_type"], axis=1)
# Data_OOT2 =Data_OOT2.drop(["kissht_user_id","data_type"], axis=1)
print(Data_train.shape)
print(Data_test.shape)
print(Data_OOT.shape)
# print(Data_OOT2.shape)

(24409, 74)
(12022, 74)
(7731, 74)


 # PRE MODEL BUILDING ANALYSIS 
 
<br> Pre-Model Building Analysis is the data analysis to check the population variance and distribution thorughout the data set 
1. **CSI** **(charterstics stabability index)** 
<br>i) It is value that is estimated for the variables for train_test and train_oot. 
<br>ii) CSI greater 0.2 indicates unstablility of variable and below 0.1 is stability
<br>iii) CSI is estimated by using PSI (https://www.listendata.com/2015/05/population-stability-index.html) for charterstics 
2. **Count of NA's / Zero distribution**
<br>i) The distribution NA's and zero is important for checking data sanity.
<br>ii) Here we estimated the number of NA's/ Zero in train , test and oot.
<br>iii)As a thumb rule the difference between train, test and oot should'nt be more than 15%. 
2. **Basic statistics**
<br>i) It basically includes mean , standard deviation, median , quartiles
<br>ii) Here we estimate and compare the basic statistics between train , test and oot.


In [82]:
# CSI
def variable_csi(df1,df2,variable,bins):
        
        quntiles = df1.quantile(list(np.array(range(0,105,int(100/bins)))/100), axis = 0)
        quant_list = sorted(list(set(list(quntiles[variable].values))))  # Remove duplicate quntiles
        labels = list(range(1,len(quant_list)))
        distiribution_1 = np.array(pd.cut(x=df1[variable], bins=quant_list, labels=labels).value_counts(sort=False))/df1.shape[0]
        distiribution_2 = np.array(pd.cut(x=df2[variable], bins=quant_list, labels=labels).value_counts(sort=False))/df2.shape[0]
        diff_distribution = distiribution_1 - distiribution_2
        log_div_distribution = np.log(distiribution_1/distiribution_2)
        psi_dist = diff_distribution*log_div_distribution
        csi = str(np.sum(psi_dist))
        
        CSI = pd.DataFrame()
        CSI['n_tiles'] = labels
        CSI['score_bands'] = quant_list[1:]
        CSI['array_1'] = distiribution_1
        CSI['array_2'] = distiribution_2
        CSI['diff_array1_array_2'] = diff_distribution
        CSI['log_div_distribution'] = log_div_distribution
        CSI['Index'] = psi_dist
        CSI['csi'] = str(np.sum(psi_dist))
        CSI['Var'] = variable
        
        return CSI

train_test = pd.DataFrame()
for column in list(Data_train.columns):
    csis = variable_csi(Data_train,Data_test,column,20)
    train_test = train_test.append(csis,ignore_index=True)

#train_test.to_csv("C:/Users/Shawn/Downloads/Variable_wise_CSI_distribution_train_test.csv",index=False)

train_oot = pd.DataFrame()
for column in list(Data_train.columns):
    csis = variable_csi(Data_train,Data_OOT,column,20)
    train_oot = train_oot.append(csis,ignore_index=True)

#train_oot.to_csv("C:/Users/Shawn/Downloads/Variable_wise_CSI_distribution_train_oot.csv",index=False)

In [83]:
# column

In [84]:
# Count of zero
def count_zero_na_empty_values(input_data_1,input_data_2, input_data_3):
    selected_columns = input_data_1.columns
    
    input_data_1 =input_data_1[selected_columns].apply(pd.to_numeric, errors ='coerce')
    input_data_2 =input_data_2[selected_columns].apply(pd.to_numeric, errors ='coerce')
    input_data_3 =input_data_3[selected_columns].apply(pd.to_numeric, errors ='coerce')
    
    input_data_1 =input_data_1.fillna(0) 
    input_data_2 =input_data_2.fillna(0)
    input_data_3 =input_data_3.fillna(0)
    
    df =pd.DataFrame()
    
    for col_values in selected_columns:
        count_of_zero_na_null_values_1=len(input_data_1[input_data_1[col_values] == 0.0])
        count_of_zero_na_null_values_2 =len(input_data_2[input_data_2[col_values] == 0.0])
        count_of_zero_na_null_values_3 =len(input_data_3[input_data_3[col_values] == 0.0])
        
        percent_of_zero_na_null_values_1=len(input_data_1[input_data_1[col_values] == 0.0])*100/len(input_data_1)
        percent_of_zero_na_null_values_2 =len(input_data_2[input_data_2[col_values] == 0.0])*100/len(input_data_2)
        percent_of_zero_na_null_values_3 =len(input_data_3[input_data_3[col_values] == 0.0])*100/len(input_data_3)
        
        df =df.append({'variable_name': col_values, 
                       'train_count_na/zero': count_of_zero_na_null_values_1,
                       'test_count_na/zero': count_of_zero_na_null_values_2,
                       'oot_count_na/zero': count_of_zero_na_null_values_3,
                      'train_percent_na/zero': percent_of_zero_na_null_values_1,
                       'test_percent_na/zero': percent_of_zero_na_null_values_2,
                      'oot_percent_na/zero': percent_of_zero_na_null_values_3},
                      ignore_index =True)
        
    
    
    #df=df.sort_values(['Difference'], ascending =False)

    return(df)
    

In [85]:
#basic_statistics
def basic_statistics(df_train, df_test, df_oot):
    
    train_stat =df_train.describe().T
    test_stat =df_test.describe().T
    oot_stat =df_oot.describe().T
    
    train_stat.columns =train_stat.columns +str('train_')
    test_stat.columns = test_stat.columns +str('test_')
    oot_stat.columns = oot_stat.columns +str('oot_')
    
    basic_stat =pd.concat([train_stat,test_stat,oot_stat], axis =1)
    return(basic_stat)

basic_statistics_train_test_oot =basic_statistics(Data_train, Data_test, Data_OOT)


In [86]:
# PRE ANALYSIS Report
writer = pd.ExcelWriter('PRE_ANALYSIS.xlsx', engine='xlsxwriter')
train_test.to_excel(writer, sheet_name='Variable_CSI_train_test', index=False)
train_oot.to_excel(writer, sheet_name='Variable_CSI_train_oot', index=False)
count_zero_na_empty_values(Data_train,Data_test,Data_OOT).to_excel(writer, sheet_name='PERCENT_ZEROS', index=False)
basic_statistics_train_test_oot[basic_statistics_train_test_oot.columns.sort_values()].to_excel(writer, sheet_name='Basic_Statistics')

writer.save()
writer.close()

In [87]:
Data_train_user_ids.to_csv('Data_train_user_ids.csv', index=False)
Data_test_user_ids.to_csv('Data_test_user_ids.csv', index=False)
Data_OOT_user_ids.to_csv('Data_OOT_user_ids.csv', index=False)

# XGBoost Model 

Here we train and create a inital set of model were we using all variables for training.Here we extract the top 30/20 variables that will be further used to create the final model 

1. **Hyper** **parameter** **grid** **search**
<br>i) We estimate the best parameteres for Xgboost using  random grid search ( optimization search) and based on startified k-fold validation
2. **Top 30 variable selection**
<br> We select top 30 variables using total_gain for the model from variables.


In [88]:
from xgboost import XGBClassifier
import xgboost as xgb 
from sklearn.metrics import accuracy_score

In [89]:
y_train =Data_train["t_flag"]
x_train =Data_train[numeric_cols].fillna(0)
#test data
y_test =Data_test["t_flag"]
x_test =Data_test[numeric_cols].fillna(0)
#Out of Time
y_oot =Data_OOT["t_flag"]
x_oot =Data_OOT[numeric_cols].fillna(0)
##Out of Time2
#y_oot2 =Data_OOT2["t_10"]
#x_oot2 =Data_OOT2[numeric_cols].fillna(0)
print(Data_train.shape)
print(x_train.shape)
print(x_train.columns)

(24409, 74)
(24409, 73)
Index(['avg_12_4_debit_count', 'avg_3_mon_income', 'avg_3_month_bounce_count',
       'avg_6_mon_income', 'avg_6_month_bounce_count', 'avg_9_12_debit_amount',
       'avg_9_12_debit_count', 'avg_9_12_debit_amount_per_count',
       'avg_balance_3_month', 'avg_balance_6_month',
       'avg_cashwithdrawl_6_month', 'avg_cc_spend_count',
       'avg_diff_credit_debit', 'count_loan_overdues',
       'count_loan_overdues_1m', 'count_name_match', 'credit_sum_avg',
       'debit_sum_avg', 'diff_credit_debit', 'estimated_income_v2',
       'kreditb_loan_approval_before_fa', 'kreditb_loan_overdue_before_fa',
       'kreditbee_sum_paid_loans', 'last_mon_12_4_debit_count',
       'last_mon_9_12_debit_amount', 'last_mon_income',
       'last_month_bounce_count', 'loan_overdues', 'loan_overdues_1m',
       'month_of_max_balance', 'number_of_application', 'number_of_devices_2w',
       'primary_bank_avg_balance', 'primary_bank_avg_balance_3_mon',
       'primary_bank_avg_balan

In [90]:
# list(x_train.dtypes)

In [91]:
numeric_cols

['avg_12_4_debit_count',
 'avg_3_mon_income',
 'avg_3_month_bounce_count',
 'avg_6_mon_income',
 'avg_6_month_bounce_count',
 'avg_9_12_debit_amount',
 'avg_9_12_debit_count',
 'avg_9_12_debit_amount_per_count',
 'avg_balance_3_month',
 'avg_balance_6_month',
 'avg_cashwithdrawl_6_month',
 'avg_cc_spend_count',
 'avg_diff_credit_debit',
 'count_loan_overdues',
 'count_loan_overdues_1m',
 'count_name_match',
 'credit_sum_avg',
 'debit_sum_avg',
 'diff_credit_debit',
 'estimated_income_v2',
 'kreditb_loan_approval_before_fa',
 'kreditb_loan_overdue_before_fa',
 'kreditbee_sum_paid_loans',
 'last_mon_12_4_debit_count',
 'last_mon_9_12_debit_amount',
 'last_mon_income',
 'last_month_bounce_count',
 'loan_overdues',
 'loan_overdues_1m',
 'month_of_max_balance',
 'number_of_application',
 'number_of_devices_2w',
 'primary_bank_avg_balance',
 'primary_bank_avg_balance_3_mon',
 'primary_bank_avg_balance_6_mon',
 'primary_bank_max_balance',
 'primary_bank_mon_max_balance',
 'six_mon_avg_12_4_de

In [92]:
x_train_v2, x_test_v2 , y_train_v2 , y_test_v2 =train_test_split(x_train, y_train, test_size =0.25)
print(x_train.shape)
print(x_train_v2.shape)


(24409, 73)
(18306, 73)


In [93]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [30,40,50,60],
        'learning_rate': [0.01,0.02,0.03,0.04,0.05,0.06],
        'gamma': [4, 5, 5.5, 6],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.4,0.5,0.6, 0.7, 0.8, 0.9, 1.0],
        'max_depth': [4, 5, 6, 7],
        'n_estimators':[50,100,200,250,300,400]
        }

In [94]:
xgb = XGBClassifier(learning_rate=0.03, objective='binary:logistic',tree_method ='exact',
                    silent=True, nthread=1)

**Hyperparameter** **Grid** **Search**

In [95]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [96]:
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = False, random_state = 42)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=-1, cv=skf.split(x_train_v2, y_train_v2), verbose=3, random_state=42)

start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(x_train_v2, y_train_v2)
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  2.3min remaining:   43.4s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 43.8min finished



 Time taken: 0 hours 44 minutes and 7.37 seconds.


In [97]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best AUC')
print(random_search.best_score_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([47.85669456, 44.47514238, 85.66282916, 12.3730093 , 42.75736547]), 'std_fit_time': array([0.34430225, 2.17616563, 0.97690561, 0.44894789, 6.48521703]), 'mean_score_time': array([0.1428411 , 0.18222375, 0.1750483 , 0.04406404, 0.11489382]), 'std_score_time': array([0.00649412, 0.02483981, 0.02023162, 0.00567051, 0.04709092]), 'param_subsample': masked_array(data=[0.8, 1.0, 0.9, 1.0, 0.9],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[250, 300, 400, 50, 400],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[40, 60, 60, 40, 50],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[7, 7, 5, 5, 4],
             mask=[False, False, False, False, False],
       fill_v

In [98]:
#select the above model from Best estimator 
xgb_clf =XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=5.5,
              learning_rate=0.04, max_delta_step=0, max_depth=4,
              min_child_weight=50, missing=None, n_estimators=400, n_jobs=1,
              nthread=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.9, tree_method='exact', verbosity=1)

# xgb_clf1 = random_search.best_estimator_

In [99]:
xgb_clf.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=5.5,
              learning_rate=0.04, max_delta_step=0, max_depth=4,
              min_child_weight=50, missing=None, n_estimators=400, n_jobs=1,
              nthread=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.9, tree_method='exact', verbosity=1)

In [100]:
# train, test and oot auc
y_pro_train_v2 =xgb_clf.predict_proba(x_train_v2)[:, 1]
train_pro_v2 =metrics.roc_auc_score(y_train_v2,y_pro_train_v2)
print(train_pro_v2)
y_pro_test_v2 =xgb_clf.predict_proba(x_test_v2)[:, 1]
test_pro_v2 =metrics.roc_auc_score(y_test_v2,y_pro_test_v2)
print(test_pro_v2)
y_pro_test =xgb_clf.predict_proba(x_test)[:, 1]
test_pro =metrics.roc_auc_score(y_test,y_pro_test)
print(test_pro)
y_pro_oot =xgb_clf.predict_proba(x_oot)[:, 1]
oot_pro =metrics.roc_auc_score(y_oot, y_pro_oot)
print(oot_pro)

0.7095275195818062
0.7122370199285055
0.629006169766159
0.617810785082628


In [101]:
print(Data_train.groupby(['t_flag'])['age'].count()/ (Data_train.groupby(['t_flag'])['age'].count()[1]+Data_train.groupby(['t_flag'])['age'].count()[0]))
print(Data_test.groupby(['t_flag'])['age'].count()/ (Data_test.groupby(['t_flag'])['age'].count()[1]+Data_test.groupby(['t_flag'])['age'].count()[0]))
print(Data_OOT.groupby(['t_flag'])['age'].count()/ (Data_OOT.groupby(['t_flag'])['age'].count()[1]+Data_OOT.groupby(['t_flag'])['age'].count()[0]))


t_flag
0.0    0.445532
1.0    0.554468
Name: age, dtype: float64
t_flag
0.0    0.385127
1.0    0.614873
Name: age, dtype: float64
t_flag
0.0    0.4015
1.0    0.5985
Name: age, dtype: float64


In [102]:
xgb_fea_imp=pd.DataFrame(list(xgb_clf.get_booster().get_score(importance_type="gain").items()),
columns=['feature','importance']).sort_values('importance', ascending=False)
print('',xgb_fea_imp)
xgb_fea_imp.to_csv('xgb_fea_imp_digi_delinquent_gain.csv')

                      feature  importance
11        avg_cc_spend_count   50.972982
0               total_income   30.584187
12           last_mon_income   23.475118
8           avg_6_mon_income   21.977959
1             total_contacts   21.220134
..                       ...         ...
45     last_six_month_max_pl    6.667912
70  kreditbee_sum_paid_loans    6.560516
60     is_digiscore_reliable    6.131992
62   three_months_emi_sum_pl    5.777320
48             is_wonderland    5.635614

[73 rows x 2 columns]


In [103]:
xgb_fea_imp=pd.DataFrame(list(xgb_clf.get_booster().get_score(importance_type="cover").items()),
columns=['feature','importance']).sort_values('importance', ascending=False)
print('',xgb_fea_imp)
xgb_fea_imp.to_csv('xgb_fea_imp_digi_delinquent_cover.csv')

                            feature   importance
56        avg_3_month_bounce_count  4321.705262
57  kreditb_loan_overdue_before_fa  4312.827911
11              avg_cc_spend_count  3895.206303
60           is_digiscore_reliable  3691.214009
59             total_loan_rejected  3373.143118
..                             ...          ...
42           avg_9_12_debit_amount  1173.176913
46    twelve_month_sum_emi_overall  1042.959507
68            number_of_devices_2w  1039.138764
6             avg_12_4_debit_count   862.959211
71   total_loan_approval_before_fa   473.011666

[73 rows x 2 columns]


In [104]:
xgb_fea_imp=pd.DataFrame(list(xgb_clf.get_booster().get_score(importance_type="total_gain").items()),
columns=['feature','importance']).sort_values('importance', ascending=False)
print('',xgb_fea_imp)
xgb_fea_imp.to_csv('xgb_fea_imp_digi_delinquent_total_gain.csv')

                           feature   importance
11             avg_cc_spend_count  3873.946618
1                  total_contacts  2567.636180
0                    total_income  2232.645654
12                last_mon_income  2018.860187
25   customer_first_name_messages  1739.978363
..                            ...          ...
71  total_loan_approval_before_fa    37.855554
61      last_mon_12_4_debit_count    36.155596
48                  is_wonderland    33.813684
72            count_loan_overdues    20.264703
68           number_of_devices_2w    16.758444

[73 rows x 2 columns]


In [105]:
xgb_fea_imp=pd.DataFrame(list(xgb_clf.get_booster().get_fscore().items()),
columns=['feature','importance']).sort_values('importance', ascending=False)
print('',xgb_fea_imp)
xgb_fea_imp.to_csv('xgb_fea_imp_digi_delinquent_f_score.csv')

                           feature  importance
25   customer_first_name_messages         166
23             total_credit_count         149
4      device_present_age_in_days         129
28          number_of_application         125
1                  total_contacts         121
..                            ...         ...
48                  is_wonderland           6
61      last_mon_12_4_debit_count           5
71  total_loan_approval_before_fa           5
72            count_loan_overdues           3
68           number_of_devices_2w           2

[73 rows x 2 columns]


In [106]:
top_xgb_vars = xgb_fea_imp.feature[0:38]
print(top_xgb_vars)

25        customer_first_name_messages
23                  total_credit_count
4           device_present_age_in_days
28               number_of_application
1                       total_contacts
2                  avg_balance_3_month
13                 avg_balance_6_month
12                     last_mon_income
53                                 age
69     avg_9_12_debit_amount_per_count
11                  avg_cc_spend_count
0                         total_income
14                 estimated_income_v2
8                     avg_6_mon_income
31          last_mon_9_12_debit_amount
19              last_month_max_overall
21                   total_debit_count
49            primary_bank_max_balance
15                    avg_3_mon_income
10                avg_9_12_debit_count
51              avg_twelve_monthly_emi
43                    count_name_match
55           avg_cashwithdrawl_6_month
41      primary_bank_avg_balance_3_mon
39                       debit_sum_avg
5                       c

###### Final Model with top 46 variables
This will be our Final model. Here we use top 20/30 variables for creating the model. Following are the things to be noted while creation
1. Data Preparation
<br>i) We will be selecting top 20 varibales for training.Here one must take care to avoid substitute NA's ( in general it's zero).
<br>ii)It is good practice to sorted columns in ascending order with their names and train the same
2. Model Creating
<br>i) We must avoid overfitting the model this can figured by checking the auc on train, test and oot. If the oot auc is much lower than training and testing then it is indication of overfitting.
Overfitting can be reduced by reducing n_estimators (generally 300  for stable) or max_dept (generally 4 for stable)
<br>ii) Normally the difference between the auc of train and oot is close to 0.4 but if the difference is more than 0.6 then their is either case of overfitting or the variables distribution in data type. The variable can be checked in PRE-ANALYSIS
 

In [107]:
Data_v1 =pd.read_csv("Covid_data.csv")
print(len(All_data))
Data_v1['r_rows'] =np.where((Data_v1['data_type'] =='Train') &(Data_v1['t_flag'] ==1) & (Data_v1['t_10'] ==0),0,1)
Data_v1 =Data_v1[Data_v1['r_rows'] ==1].reset_index(drop =True)
Data_v1 = Data_v1.dropna(subset=['t_flag'], axis =0).reset_index(drop =True)
Data_v1.columns = Data_v1.columns.str.replace(".","_")
Data_v1.columns = Data_v1.columns.str.lower()
print(Data_v1.shape)

44162
(44162, 224)


In [108]:
retained_columns = ['kissht_user_id','t_flag','data_type']
retained_columns.extend(top_xgb_vars)
# Data_v1.rename(columns={'age': 'Age'},inplace = True)


In [109]:
print(Data_v1.shape)
Data_v1 =Data_v1[retained_columns]
print(Data_v1.shape)

(44162, 224)
(44162, 41)


In [110]:
numeric_cols = top_xgb_vars.tolist()

In [111]:
Data_v1[numeric_cols] = Data_v1[numeric_cols].apply(pd.to_numeric, errors='coerce')
Data_v1[numeric_cols] = Data_v1[numeric_cols].fillna(0)

Data_v1.head()

,kissht_user_id,t_flag,data_type,customer_first_name_messages,total_credit_count,device_present_age_in_days,number_of_application,total_contacts,avg_balance_3_month,avg_balance_6_month,...,last_six_month_count_loans_pl,last_twelve_month_max_overall,avg_9_12_debit_amount,total_loan_rejected,kreditb_loan_overdue_before_fa,primary_bank_avg_balance,three_month_sum_overdue_overall,twelve_month_sum_overdue_overall,six_month_sum_overdue_overall,avg_six_monthly_emi
0,3530575,1.0,Train,177,41,12,34,284,2797.00000,2797.00,...,1.0,2774.0,6685.0,0,0,3762.000,836.0,836.0,836.0,2774.0
1,8334491,0.0,Train,830,8,365,55,156,992.50000,63685.25,...,0.0,11303.0,144100.0,0,0,14059.600,0.0,0.0,0.0,0.0
2,8329352,1.0,Train,279,11,363,36,374,10484.33333,8856.50,...,1.0,3999.0,4697.0,0,0,4163.875,0.0,0.0,0.0,5241.0
3,8266353,1.0,Train,98,3,12,51,124,0.00000,0.00,...,0.0,0.0,0.0,0,0,0.000,0.0,0.0,0.0,0.0
4,8286716,0.0,Train,60,0,16,32,339,0.00000,0.00,...,0.0,0.0,4000.0,0,0,0.000,0.0,0.0,0.0,0.0


In [112]:
Data_train =Data_v1[Data_v1["data_type"]=="Train"]
Data_test =Data_v1[Data_v1["data_type"]=="Test"]
Data_OOT =Data_v1[Data_v1["data_type"]=="OOT"]
# Data_OOT2 =Data_v1[Data_v1["data_type"]=="OOT2"]
Data_train_user_ids = Data_train["kissht_user_id"]
Data_test_user_ids = Data_test["kissht_user_id"]
Data_OOT_user_ids = Data_OOT["kissht_user_id"]
# Data_OOT2_user_ids = Data_OOT2["kissht_user_id"]
Data_test =Data_test.drop(["kissht_user_id","data_type"], axis=1)
Data_train =Data_train.drop(["kissht_user_id","data_type"], axis=1)
Data_OOT =Data_OOT.drop(["kissht_user_id","data_type"], axis=1)
# Data_OOT2 =Data_OOT2.drop(["kissht_user_id","data_type"], axis=1)
print(Data_train.shape)
print(Data_test.shape)
print(Data_OOT.shape)
# print(Data_OOT2.shape)

(24409, 39)
(12022, 39)
(7731, 39)


In [113]:
Data_train.dtypes

t_flag                              float64
customer_first_name_messages          int64
total_credit_count                    int64
device_present_age_in_days            int64
number_of_application                 int64
total_contacts                        int64
avg_balance_3_month                 float64
avg_balance_6_month                 float64
last_mon_income                     float64
age                                   int64
avg_9_12_debit_amount_per_count     float64
avg_cc_spend_count                  float64
total_income                        float64
estimated_income_v2                 float64
avg_6_mon_income                    float64
last_mon_9_12_debit_amount            int64
last_month_max_overall              float64
total_debit_count                     int64
primary_bank_max_balance              int64
avg_3_mon_income                    float64
avg_9_12_debit_count                float64
avg_twelve_monthly_emi              float64
count_name_match                

In [114]:
from xgboost import XGBClassifier
import xgboost as xgb 
from sklearn.metrics import accuracy_score

In [115]:
y_train =Data_train["t_flag"]
x_train =Data_train[numeric_cols]
#test data
y_test =Data_test["t_flag"]
x_test =Data_test[numeric_cols]
#Out of Time
y_oot =Data_OOT["t_flag"]
x_oot =Data_OOT[numeric_cols]
#Out of Time2
#y_oot2 =Data_OOT2["t_10"]
#x_oot2 =Data_OOT2[numeric_cols]
print(Data_train.shape)
print(x_train.shape)

(24409, 39)
(24409, 38)


In [116]:
x_train_v2, x_test_v2 , y_train_v2 , y_test_v2 =train_test_split(x_train, y_train, test_size =0.25 , random_state = 42)
print(x_train.shape)
print(x_train_v2.shape)

(24409, 38)
(18306, 38)


In [117]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [2,5,10,20,30,40,50],
        'learning_rate': [0.01,0.02,0.03,0.04,0.5,1,5,10],
        'gamma': [5,6,7, 8,9, 10],
        'subsample': [0.4, 0.6, 0.8, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0],
        'max_depth': [4, 5, 6, 7],
        'n_estimators':[80,100,160,200,250,300]
        }

In [118]:
xgb = XGBClassifier(learning_rate=0.03, objective='binary:logistic',tree_method ='exact',
                    silent=True, nthread=1)

In [119]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [120]:
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = False, random_state = 42)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=-1, cv=skf.split(x_train_v2, y_train_v2), verbose=3, random_state=42)

start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(x_train_v2, y_train_v2)
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  1.5min remaining:   27.9s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  1.6min finished



 Time taken: 0 hours 2 minutes and 11.14 seconds.


In [121]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best AUC')
print(random_search.best_score_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-02.csv', index=False)


 All results:
{'mean_fit_time': array([ 3.1529727 , 30.97102437, 38.92955456, 54.28672657, 11.96352315]), 'std_fit_time': array([0.23889539, 0.1202101 , 4.13160697, 0.41075364, 0.9158528 ]), 'mean_score_time': array([0.02194171, 0.14261885, 0.14581046, 0.14142737, 0.05644913]), 'std_score_time': array([0.0006306 , 0.01244086, 0.01791195, 0.01586961, 0.00794355]), 'param_subsample': masked_array(data=[1.0, 0.4, 0.4, 0.8, 0.6],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[80, 300, 300, 300, 80],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[2, 2, 5, 30, 10],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[6, 5, 5, 6, 5],
             mask=[False, False, False, False, False],
       fill_value

In [122]:
xgb_clf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=5,
              learning_rate=0.02, max_delta_step=0, max_depth=5,
              min_child_weight=2, missing=None, n_estimators=300, n_jobs=1,
              nthread=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.4, tree_method='exact', verbosity=1)



In [123]:
x_train =x_train[sorted(x_train.columns)]

In [124]:
eval_set =[(x_oot[sorted(x_train.columns)], y_oot)]
xgb_clf.fit(x_train,y_train,eval_set =eval_set ,eval_metric=["auc","error","logloss"])

[0]	validation_0-auc:0.583433	validation_0-error:0.390247	validation_0-logloss:0.692012
[1]	validation_0-auc:0.591322	validation_0-error:0.395292	validation_0-logloss:0.690948
[2]	validation_0-auc:0.597222	validation_0-error:0.394386	validation_0-logloss:0.689923
[3]	validation_0-auc:0.59841	validation_0-error:0.394386	validation_0-logloss:0.688959
[4]	validation_0-auc:0.599606	validation_0-error:0.391799	validation_0-logloss:0.688029
[5]	validation_0-auc:0.599532	validation_0-error:0.390894	validation_0-logloss:0.687149
[6]	validation_0-auc:0.60068	validation_0-error:0.386108	validation_0-logloss:0.686273
[7]	validation_0-auc:0.601192	validation_0-error:0.388566	validation_0-logloss:0.685475
[8]	validation_0-auc:0.602335	validation_0-error:0.390247	validation_0-logloss:0.684642
[9]	validation_0-auc:0.60212	validation_0-error:0.391153	validation_0-logloss:0.683903
[10]	validation_0-auc:0.602639	validation_0-error:0.395033	validation_0-logloss:0.683198
[11]	validation_0-auc:0.604146	val

[93]	validation_0-auc:0.608498	validation_0-error:0.390118	validation_0-logloss:0.662849
[94]	validation_0-auc:0.608737	validation_0-error:0.38973	validation_0-logloss:0.662755
[95]	validation_0-auc:0.608941	validation_0-error:0.390764	validation_0-logloss:0.662688
[96]	validation_0-auc:0.608924	validation_0-error:0.390118	validation_0-logloss:0.662668
[97]	validation_0-auc:0.608932	validation_0-error:0.390247	validation_0-logloss:0.662675
[98]	validation_0-auc:0.608952	validation_0-error:0.389988	validation_0-logloss:0.66269
[99]	validation_0-auc:0.608828	validation_0-error:0.38973	validation_0-logloss:0.662691
[100]	validation_0-auc:0.608827	validation_0-error:0.3896	validation_0-logloss:0.66264
[101]	validation_0-auc:0.608924	validation_0-error:0.3896	validation_0-logloss:0.662591
[102]	validation_0-auc:0.608903	validation_0-error:0.390247	validation_0-logloss:0.662539
[103]	validation_0-auc:0.609083	validation_0-error:0.390376	validation_0-logloss:0.662468
[104]	validation_0-auc:0.

[185]	validation_0-auc:0.611755	validation_0-error:0.387272	validation_0-logloss:0.661423
[186]	validation_0-auc:0.611797	validation_0-error:0.386625	validation_0-logloss:0.661417
[187]	validation_0-auc:0.611752	validation_0-error:0.386108	validation_0-logloss:0.661465
[188]	validation_0-auc:0.611736	validation_0-error:0.386625	validation_0-logloss:0.661491
[189]	validation_0-auc:0.611705	validation_0-error:0.387013	validation_0-logloss:0.661518
[190]	validation_0-auc:0.611643	validation_0-error:0.388177	validation_0-logloss:0.661566
[191]	validation_0-auc:0.611657	validation_0-error:0.387272	validation_0-logloss:0.661551
[192]	validation_0-auc:0.611732	validation_0-error:0.388177	validation_0-logloss:0.661576
[193]	validation_0-auc:0.611707	validation_0-error:0.388177	validation_0-logloss:0.661603
[194]	validation_0-auc:0.611665	validation_0-error:0.388436	validation_0-logloss:0.661619
[195]	validation_0-auc:0.611896	validation_0-error:0.388824	validation_0-logloss:0.661586
[196]	vali

[277]	validation_0-auc:0.612572	validation_0-error:0.389471	validation_0-logloss:0.66229
[278]	validation_0-auc:0.612629	validation_0-error:0.390247	validation_0-logloss:0.662292
[279]	validation_0-auc:0.612715	validation_0-error:0.389471	validation_0-logloss:0.66224
[280]	validation_0-auc:0.612662	validation_0-error:0.389083	validation_0-logloss:0.662239
[281]	validation_0-auc:0.61269	validation_0-error:0.389342	validation_0-logloss:0.662303
[282]	validation_0-auc:0.612574	validation_0-error:0.389471	validation_0-logloss:0.662353
[283]	validation_0-auc:0.612625	validation_0-error:0.3896	validation_0-logloss:0.662348
[284]	validation_0-auc:0.612644	validation_0-error:0.389212	validation_0-logloss:0.662353
[285]	validation_0-auc:0.612625	validation_0-error:0.388824	validation_0-logloss:0.662367
[286]	validation_0-auc:0.612673	validation_0-error:0.388566	validation_0-logloss:0.662394
[287]	validation_0-auc:0.612676	validation_0-error:0.388824	validation_0-logloss:0.662384
[288]	validatio

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=5,
              learning_rate=0.02, max_delta_step=0, max_depth=5,
              min_child_weight=2, missing=None, n_estimators=300, n_jobs=1,
              nthread=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=0.4, tree_method='exact', verbosity=1)

In [125]:
# train, test and oot auc
y_pro_train_v2 =xgb_clf.predict_proba(x_train_v2[sorted(x_train.columns)])[:, 1]
train_pro_v2 =metrics.roc_auc_score(y_train_v2,y_pro_train_v2)
print(train_pro_v2)
y_pro_test_v2 =xgb_clf.predict_proba(x_test_v2[sorted(x_train.columns)])[:, 1]
test_pro_v2 =metrics.roc_auc_score(y_test_v2,y_pro_test_v2)
print(test_pro_v2)
y_pro_train =xgb_clf.predict_proba(x_train[sorted(x_train.columns)])[:, 1]
train_pro =metrics.roc_auc_score(y_train,y_pro_train)
print(train_pro)
y_pro_test =xgb_clf.predict_proba(x_test[sorted(x_train.columns)])[:, 1]
test_pro =metrics.roc_auc_score(y_test,y_pro_test)
print(test_pro)
y_pro_oot =xgb_clf.predict_proba(x_oot[sorted(x_train.columns)])[:, 1]
oot_pro =metrics.roc_auc_score(y_oot, y_pro_oot)
print(oot_pro)

0.7143205235074567
0.7205562372267755
0.7159120342296711
0.6248270852617505
0.6126819427764868


In [126]:
x_train.columns

Index(['age', 'avg_3_mon_income', 'avg_6_mon_income', 'avg_9_12_debit_amount',
       'avg_9_12_debit_amount_per_count', 'avg_9_12_debit_count',
       'avg_balance_3_month', 'avg_balance_6_month',
       'avg_cashwithdrawl_6_month', 'avg_cc_spend_count',
       'avg_diff_credit_debit', 'avg_six_monthly_emi',
       'avg_twelve_monthly_emi', 'count_name_match', 'credit_sum_avg',
       'customer_first_name_messages', 'debit_sum_avg',
       'device_present_age_in_days', 'diff_credit_debit',
       'estimated_income_v2', 'kreditb_loan_overdue_before_fa',
       'last_mon_9_12_debit_amount', 'last_mon_income',
       'last_month_max_overall', 'last_six_month_count_loans_pl',
       'last_twelve_month_max_overall', 'number_of_application',
       'primary_bank_avg_balance', 'primary_bank_avg_balance_3_mon',
       'primary_bank_max_balance', 'six_month_sum_overdue_overall',
       'three_month_sum_overdue_overall', 'total_contacts',
       'total_credit_count', 'total_debit_count', 'total

In [127]:
Xgboostmodel  = "Xgboostmodel_non_repeat_30_04_2020_1233"
xgb_clf._Booster.save_model(Xgboostmodel)

In [128]:
print(Data_train.shape)
y_pro_train =xgb_clf.predict_proba(x_train)[:, 1]

(24409, 39)


# POST MODEL BUILDING ANALYSIS
Here, we check the model stability. This is done by using PSI calculation 
1.Population Stability Index
<br> i) PSI score gives us the model stability indication. It is calculated between Train_Test_data and Train_OOT data. Lower the PSI stable is the model
<br> a) PSI score < 0.1 - The model is stable
<br> b) PSI score > 0.2 - The model is unstable and should'nt be used 
<br> c) PSI score >=0.1 and <0.2 - The model is bit stable and can be corrected by fine tuning

<br> https://www.listendata.com/2015/05/population-stability-index.html - for reference

In [129]:
print(Data_train.shape)
print(Data_test.shape)
print(Data_OOT.shape)
#print(Data_OOT2.shape)
Data_train_final = Data_train
Data_test_final = Data_test
Data_OOT_final = Data_OOT
#Data_OOT2_final = Data_OOT2
Data_train_final['predicted_vals'] = y_pro_train.tolist()
Data_test_final['predicted_vals'] = y_pro_test.tolist()
Data_OOT_final['predicted_vals'] = y_pro_oot.tolist()
#Data_OOT2_final['predicted_vals'] = y_pro_oot2.tolist()
print(Data_train_final.shape)
print(Data_test_final.shape)
print(Data_OOT_final.shape)
#print(Data_OOT2_final.shape)

(24409, 39)
(12022, 39)
(7731, 39)
(24409, 40)
(12022, 40)
(7731, 40)


In [130]:
Data_train_final['user_id']=Data_train_user_ids.tolist()
Data_test_final['user_id']=Data_test_user_ids.tolist()
Data_OOT_final['user_id']=Data_OOT_user_ids.tolist()
Data_train_final.to_csv('Data_train_final.csv', index=False)
Data_test_final.to_csv('Data_test_final.csv', index=False)
Data_OOT_final.to_csv('Data_OOT_final.csv', index=False)

 **N** **Tiles** **Distribution**

In [131]:
def n_tile(df1,df2,df3,bins):
    quantiles = list(df1.quantile(list(np.array(range(0,105,int(100/bins)))/100)))
    
    labels = list(range(1,bins+1,1))
    distiribution_1 = np.array(pd.cut(x=df1, bins=quantiles, labels=labels).value_counts(sort=False))/df1.shape[0]
    distiribution_2 = np.array(pd.cut(x=df2, bins=quantiles, labels=labels).value_counts(sort=False))/df2.shape[0]
    distiribution_3 = np.array(pd.cut(x=df3, bins=quantiles, labels=labels).value_counts(sort=False))/df3.shape[0]
    
    distribution = pd.DataFrame()
    distribution['bands'] = labels
    distribution['cuts'] = quantiles[1:]
    distribution['train'] = distiribution_1
    distribution['test'] = distiribution_2
    distribution['oot'] = distiribution_3
    return distribution

In [132]:
a = n_tile(Data_train_final['predicted_vals'],Data_test_final['predicted_vals'],Data_OOT_final['predicted_vals'],20)
b = n_tile(Data_train_final['predicted_vals'],Data_test_final['predicted_vals'],Data_OOT_final['predicted_vals'],10)
writer = pd.ExcelWriter('Bands.xlsx', engine='xlsxwriter')
a.to_excel(writer, sheet_name='N_Tile',index=False)
b.to_excel(writer, sheet_name='Decile',index=False)


In [133]:
########### Making the n_tiles for the predicted values ###########
## BANDS ##
def band(x):
    if(x<a['cuts'][0]):
        return 1
    elif(x>=a['cuts'][0] and x<a['cuts'][1]):
        return 2
    elif(x>=a['cuts'][1] and x<a['cuts'][2]):
        return 3
    elif(x>=a['cuts'][2] and x<a['cuts'][3]):
        return 4
    elif(x>=a['cuts'][3] and x<a['cuts'][4]):
        return 5
    elif(x>=a['cuts'][4] and x<a['cuts'][5]):
        return 6
    elif(x>=a['cuts'][5] and x<a['cuts'][6]):
        return 7
    elif(x>=a['cuts'][6] and x<a['cuts'][7]):
        return 8
    elif(x>=a['cuts'][7] and x<a['cuts'][8]):
        return 9
    elif(x>=a['cuts'][8] and x<a['cuts'][9]):
        return 10
    elif(x>=a['cuts'][9] and x<a['cuts'][10]):
        return 11
    elif(x>=a['cuts'][10] and x<a['cuts'][11]):
        return 12
    elif(x>=a['cuts'][11] and x<a['cuts'][12]):
        return 13
    elif(x>=a['cuts'][12] and x<a['cuts'][13]):
        return 14
    elif(x>=a['cuts'][13] and x<a['cuts'][14]):
        return 15
    elif(x>=a['cuts'][14] and x<a['cuts'][15]):
        return 16
    elif(x>=a['cuts'][15] and x<a['cuts'][16]):
        return 17
    elif(x>=a['cuts'][16] and x<a['cuts'][17]):
        return 18
    elif(x>=a['cuts'][17] and x<a['cuts'][18]):
        return 19
    elif(x>=a['cuts'][18] and x<a['cuts'][19]):
        return 20
    else:
        return np.nan
    
Data_test_final['Bins'] = Data_test_final.apply(lambda x:band(x['predicted_vals']),axis=1)
Data_OOT_final['Bins'] = Data_OOT_final.apply(lambda x:band(x['predicted_vals']),axis=1)

In [134]:
TEST = pd.crosstab(Data_test_final['Bins'],Data_test_final['t_flag']).reset_index()
TEST['grand_total'] = TEST[0] + TEST[1]
TEST['bad_rate'] = (TEST[1]/TEST['grand_total'])*100
TEST['population_dist'] = (TEST['grand_total']/sum(TEST['grand_total']))*100

OOT = pd.crosstab(Data_OOT_final['Bins'],Data_OOT_final['t_flag']).reset_index()
OOT['grand_total'] = OOT[0] + OOT[1]
OOT['bad_rate'] = (OOT[1]/OOT['grand_total'])*100
OOT['population_dist'] = (OOT['grand_total']/sum(OOT['grand_total']))*100

In [135]:
writer = pd.ExcelWriter('POST_ANALYSIS.xlsx', engine='xlsxwriter')

n_tile(Data_train_final['predicted_vals'],Data_test_final['predicted_vals'],Data_OOT_final['predicted_vals'],20).to_excel(writer, sheet_name='n_tile_cuts',index=False)
TEST.to_excel(writer, sheet_name='crosstab_test',index=False)
OOT.to_excel(writer, sheet_name='crosstab_OOT',index=False)

writer.save()
writer.close()

**Model** **Run** 

In [136]:
Data =pd.read_csv('Covid_data.csv')
Data[sorted(x_train.columns)] =Data[sorted(x_train.columns)].apply(pd.to_numeric, errors ='coerce').fillna(0)

In [137]:
Data['predict_prob'] =xgb_clf.predict_proba(Data[sorted(x_train.columns)])[:, 1]

In [138]:
Data['Bins'] = Data.apply(lambda x:band(x['predict_prob']),axis=1)


In [139]:
Data.to_csv('output_data.csv', index=False)

In [140]:
xgb_fea_imp

,feature,importance
25,customer_first_name_messages,166
23,total_credit_count,149
4,device_present_age_in_days,129
28,number_of_application,125
1,total_contacts,121
...,...,...
48,is_wonderland,6
61,last_mon_12_4_debit_count,5
71,total_loan_approval_before_fa,5
72,count_loan_overdues,3


In [141]:
num_col_final = sorted(x_train.columns)
k = pd.DataFrame(num_col_final)
k.to_csv(r'Final_numberic_column.csv')

In [142]:
w, h = 2, len(xgb_fea_imp);
testval = [[0 for x in range(w)] for y in range(h)] 



for i in range(20,len(xgb_fea_imp)):
 top_xgb_vars = xgb_fea_imp.feature[0:i]
 numeric_cols = top_xgb_vars
 y_train =Data_train["t_flag"]
 x_train =Data_train[numeric_cols]
 #test data
 y_test =Data_test["t_flag"]
 x_test =Data_test[numeric_cols]
 #Out of Time
 y_oot =Data_OOT["t_flag"]
 x_oot =Data_OOT[numeric_cols]

 print(i)
 xgb_clf.fit(x_train,y_train)   

 y_pro_train =xgb_clf.predict_proba(x_train)[:, 1]
 train_pro = metrics.roc_auc_score(y_train,y_pro_train)
 train_pro
 testval[i][0] = train_pro   
 print(train_pro)

 y_pro_oot = xgb_clf.predict_proba(x_test)[:, 1]
    
 y_pro_oot = xgb_clf.predict_proba(x_oot)[:, 1]
    
 y_testoot = pd.concat([y_test , y_oot])
 y_pro_testoot = np.concatenate((y_pro_test , y_pro_oot), axis=0)
 testoot_pro = metrics.roc_auc_score(y_testoot,y_pro_testoot) 
 testval[i][1] = testoot_pro
 print(testoot_pro)    
 print("/n")

20
0.7033332076388287
0.6181052808099098
/n
21
0.7025045275500272
0.6178071525102065
/n
22
0.7038731130961784
0.6180640834324189
/n
23
0.7054528959844002
0.6184279532581299
/n
24
0.7063956693147442
0.6186297122909414
/n
25
0.7081816659277869
0.6186443484136929
/n
26
0.7085328529764968
0.6185887731030472
/n
27
0.7088649344605074
0.6183286417056281
/n
28
0.709525778414177
0.6182983743543888
/n
29
0.7122803259224533
0.6194336609049291
/n
30
0.7127107582605918
0.6191404059338597
/n
31
0.7132192299003447
0.6188562413881666
/n
32
0.7137521304369243
0.620136813376235
/n
33
0.7141000222513245
0.6204412565631526
/n
34
0.7139489986054705
0.6197389163132858
/n
35
0.7164542599396326
0.6211119147979496
/n
36
0.7154546285302745
0.6205674413494899
/n
37
0.7172691747816058
0.620323505970301
/n
38
0.7161535443302436
0.6205136948818195
/n


KeyError: "['twelve_month_sum_emi_overall'] not in index"

In [ ]:
type(y_pro_test)

In [ ]:
w, h = 2, len(xgb_fea_imp);
testval = [[0 for x in range(w)] for y in range(h)] 



for i in range(20,len(xgb_fea_imp)):
 top_xgb_vars = xgb_fea_imp.feature[0:i]
 numeric_cols = top_xgb_vars
 y_train =Data_train["t_flag"]
 x_train =Data_train[numeric_cols]
 #test data
 y_test =Data_test["t_flag"]
 x_test =Data_test[numeric_cols]
 #Out of Time
 y_oot =Data_OOT["t_flag"]
 x_oot =Data_OOT[numeric_cols]

 print(i)
 xgb_clf.fit(x_train,y_train)   

 y_pro_train =xgb_clf.predict_proba(x_train)[:, 1]
 train_pro = metrics.roc_auc_score(y_train,y_pro_train)
 train_pro
 testval[i][0] = train_pro   
 print(train_pro)

 y_pro_oot = xgb_clf.predict_proba(x_oot)[:, 1]
 oot_pro = metrics.roc_auc_score(y_oot,y_pro_oot)
 oot_pro
 testval[i][1] =  oot_pro
 print(oot_pro)
 print("/n")

In [ ]:
testval 